In [33]:
import csv
import numpy as np
import nltk
from nltk.corpus import brown
from mittens import GloVe, Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('brown')

2024-06-19 15:53:40.279402: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package brown to /home/kojo/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [3]:
def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

glove_path = "../data/glove.840B.300d.txt"
pre_glove = glove2dict(glove_path)

In [23]:
sw = list(_stop_words.ENGLISH_STOP_WORDS)
brown_data = brown.words()[:200000]
brown_nonstop = [token.lower() for token in brown_data if (token.lower() not in sw)]
oov = [token for token in brown_nonstop if token not in pre_glove.keys()]

In [27]:
len(brown_data), len(brown_nonstop), len(oov)

(200000, 114753, 3554)

In [26]:
corp_vocab = list(set(oov))
brown_doc = [' '.join(brown_nonstop)]

In [30]:
cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
X = cv.fit_transform(brown_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

/home/kojo/miniconda3/envs/paper/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [31]:
coocc_ar.shape

(2186, 2186)

In [34]:
mittens_model = Mittens(n=300, max_iter=1000)

In [36]:
new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=corp_vocab,
    initial_embedding_dict= pre_glove)

2024-06-19 15:54:37.798783: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2024-06-19 15:54:38.989782: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
Iteration 1000: loss: 0.22750915586948395

In [37]:
new_embeddings.shape

(2186, 300)